In [1]:
import requests
import pandas as pd
import numpy as np
from IPython.display import display
from bs4 import BeautifulSoup
import re
from requests_futures.sessions import FuturesSession
import datetime

In [2]:
TOP_UNI_URL = "https://www.topuniversities.com"
start = datetime.datetime.now()

In [3]:
r = requests.get('https://www.topuniversities.com/sites/default/files/qs-rankings-data/357051_indicators.txt')
json = r.json()['data']
#print(json)

In [4]:
def getLink(x):
    soup = BeautifulSoup(x, 'html.parser')
    link = soup.find("a")
    return pd.Series([TOP_UNI_URL + link.get("href"),link.text],index = ['url','name'])

In [5]:
df = pd.DataFrame(json)
df = df[['region','overall_rank','location','uni']]
df_temp = df.apply(lambda x: getLink(x['uni']),axis = 1)
df = df.drop('uni',1)
df = df.join(df_temp)

display(df)

,region,overall_rank,location,url,name
0,North America,1,United States,https://www.topuniversities.com/universities/m...,Massachusetts Institute of Technology (MIT)
1,North America,2,United States,https://www.topuniversities.com/universities/s...,Stanford University
2,North America,3,United States,https://www.topuniversities.com/universities/h...,Harvard University
3,North America,4,United States,https://www.topuniversities.com/universities/c...,California Institute of Technology (Caltech)
4,Europe,5,United Kingdom,https://www.topuniversities.com/universities/u...,University of Cambridge
5,Europe,6,United Kingdom,https://www.topuniversities.com/universities/u...,University of Oxford
6,Europe,7,United Kingdom,https://www.topuniversities.com/universities/u...,UCL (University College London)
7,Europe,8,United Kingdom,https://www.topuniversities.com/universities/i...,Imperial College London
8,North America,9,United States,https://www.topuniversities.com/universities/u...,University of Chicago
9,Europe,10,Switzerland,https://www.topuniversities.com/universities/e...,ETH Zurich - Swiss Federal Institute of Techno...


In [6]:
def find_number(soup, list_dict_div):
    current_soup = soup
    for dict_div in list_dict_div:
        current_soup = current_soup.find("div",dict_div)
        if(current_soup is None):
            return 0
    return int(current_soup.text.replace(",", ""))

In [7]:
def getExtraData(x):
    #print(x)
    #r_t = requests.get(x)
    soup = BeautifulSoup(x.text, 'html.parser')

    total_faculty_int = find_number(soup, [{ "class" : "faculty-main wrapper col-md-4" },{ "class" : "total faculty"}, { "class" : "number"}])
    inter_faculty_int = find_number(soup, [{ "class" : "faculty-main wrapper col-md-4" },{ "class" : "inter faculty"}, { "class" : "number"}])
    number_students_int = find_number(soup, [{"class": "students-main wrapper col-md-4"}, { "class" : "number"}])
    international_students_int = find_number(soup, [{"class": "int-students-main wrapper col-md-4"}, { "class" : "number"}])

    return pd.DataFrame(data = [[total_faculty_int,inter_faculty_int,number_students_int,international_students_int]], columns = ['total faculty','international faculty','number of students','international students'])

def async_getExtraData(sess,x):
    return getExtraData(x)

In [8]:
df = df.head(200)

df_temp = df[['url']]

#display(df_temp)
urls = df_temp.values.tolist()
urls = [item for sublist in urls for item in sublist]
#print(urls)

session = FuturesSession(max_workers=10)
futures = []
results =[]

for url in urls:
    future = session.get(url)
    futures.append((url,future))

for url,r in futures:
    results.append((url,r.result()))

In [9]:
list_dataframe = []

for url,result in results:
    list_dataframe.append(pd.concat([getExtraData(result),pd.DataFrame(data = [url],columns=['url'])],axis = 1))
    
print('done')

done


In [10]:
df_temp = pd.concat(list_dataframe)
display(df_temp)

,total faculty,international faculty,number of students,international students,url
0,2982,1679,11067,3717,https://www.topuniversities.com/universities/m...
0,4285,2042,15878,3611,https://www.topuniversities.com/universities/s...
0,4350,1311,22429,5266,https://www.topuniversities.com/universities/h...
0,953,350,2255,647,https://www.topuniversities.com/universities/c...
0,5490,2278,18770,6699,https://www.topuniversities.com/universities/u...
0,6750,2964,19720,7353,https://www.topuniversities.com/universities/u...
0,6345,2554,31080,14854,https://www.topuniversities.com/universities/u...
0,3930,2071,16090,8746,https://www.topuniversities.com/universities/i...
0,2449,635,13557,3379,https://www.topuniversities.com/universities/u...
0,2477,1886,19815,7563,https://www.topuniversities.com/universities/e...


In [11]:
df = df.merge(df_temp, left_on='url', right_on='url', how='outer')
df = df.drop('url', 1)

In [12]:
display(df)

,region,overall_rank,location,name,total faculty,international faculty,number of students,international students
0,North America,1,United States,Massachusetts Institute of Technology (MIT),2982,1679,11067,3717
1,North America,2,United States,Stanford University,4285,2042,15878,3611
2,North America,3,United States,Harvard University,4350,1311,22429,5266
3,North America,4,United States,California Institute of Technology (Caltech),953,350,2255,647
4,Europe,5,United Kingdom,University of Cambridge,5490,2278,18770,6699
5,Europe,6,United Kingdom,University of Oxford,6750,2964,19720,7353
6,Europe,7,United Kingdom,UCL (University College London),6345,2554,31080,14854
7,Europe,8,United Kingdom,Imperial College London,3930,2071,16090,8746
8,North America,9,United States,University of Chicago,2449,635,13557,3379
9,Europe,10,Switzerland,ETH Zurich - Swiss Federal Institute of Techno...,2477,1886,19815,7563


In [13]:
def display_ratio(df):
    df_ratio = df.copy()
    
    df_ratio['ratio'] = df_ratio['total faculty'] / df_ratio['number of students']
    df_ratio = df_ratio.sort_values('ratio',ascending=False)
    display(df_ratio)
    
    df_ratio['ratio'] = df_ratio['international students'] / df_ratio['number of students']
    df_ratio = df_ratio.sort_values('ratio',ascending=False)
    display(df_ratio)

### a ,b)

In [14]:
display_ratio(df)

,region,overall_rank,location,name,total faculty,international faculty,number of students,international students,ratio
3,North America,4,United States,California Institute of Technology (Caltech),953,350,2255,647,0.422616
15,North America,16,United States,Yale University,4940,1708,12402,2469,0.398323
5,Europe,6,United Kingdom,University of Oxford,6750,2964,19720,7353,0.342292
4,Europe,5,United Kingdom,University of Cambridge,5490,2278,18770,6699,0.292488
16,North America,17,United States,Johns Hopkins University,4462,1061,16146,4105,0.276353
1,North America,2,United States,Stanford University,4285,2042,15878,3611,0.269870
0,North America,1,United States,Massachusetts Institute of Technology (MIT),2982,1679,11067,3717,0.269450
185,North America,186,United States,University of Rochester,2569,488,9636,2805,0.266604
18,North America,19,United States,University of Pennsylvania,5499,1383,20639,4250,0.266437
17,North America,18,United States,Columbia University,6189,913,25045,8105,0.247115


,region,overall_rank,location,name,total faculty,international faculty,number of students,international students,ratio
34,Europe,35,United Kingdom,London School of Economics and Political Scien...,1088,687,9760,6748,0.691393
11,Europe,12,Switzerland,Ecole Polytechnique Fédérale de Lausanne (EPFL),1695,1300,10343,5896,0.570047
7,Europe,8,United Kingdom,Imperial College London,3930,2071,16090,8746,0.543567
199,Europe,200,Netherlands,Maastricht University,1277,502,16385,8234,0.502533
46,North America,47,United States,Carnegie Mellon University,1342,425,13356,6385,0.478062
6,Europe,7,United Kingdom,UCL (University College London),6345,2554,31080,14854,0.477928
91,Europe,92,United Kingdom,University of St Andrews,1140,485,8800,4030,0.457955
41,Oceania,41,Australia,The University of Melbourne,3311,1477,42182,18030,0.427434
126,Europe,127,United Kingdom,Queen Mary University of London,1885,801,16135,6806,0.421816
25,Asia,26,Hong Kong,The University of Hong Kong,3012,2085,20214,8230,0.407144


### c)

In [15]:
display_ratio(df.groupby('location').sum())

,total faculty,international faculty,number of students,international students,ratio
location,,,,,
Russia,6709,373,30233,5098,0.221910
Denmark,11916,3904,67223,9543,0.177261
Saudi Arabia,1062,665,6040,989,0.175828
Singapore,9444,6079,58466,16168,0.161530
Malaysia,2755,655,17902,3476,0.153893
Japan,28395,2221,186222,16269,0.152479
South Korea,19851,2010,140071,16273,0.141721
Switzerland,15323,9208,109112,32995,0.140434
United Kingdom,79934,30216,583621,199426,0.136962


,total faculty,international faculty,number of students,international students,ratio
location,,,,,
Australia,22034,11382,301994,106359,0.352189
United Kingdom,79934,30216,583621,199426,0.341705
Hong Kong,10166,6296,78838,24499,0.310751
Austria,4117,1572,63446,19667,0.309980
Switzerland,15323,9208,109112,32995,0.302396
Singapore,9444,6079,58466,16168,0.276537
Canada,29317,10734,281514,73239,0.260161
New Zealand,3313,1404,48173,12439,0.258215
Ireland,2853,1171,34794,8187,0.235299


### d)

In [16]:
display_ratio(df.groupby('region').sum())

,total faculty,international faculty,number of students,international students,ratio
region,,,,,
Asia,106734,25462,807003,110100,0.132260
North America,182123,43836,1546353,292116,0.117776
Europe,223271,68267,1991280,454293,0.112124
Latin America,43126,5521,396902,34737,0.108657
Africa,1733,379,19593,3325,0.088450
Oceania,25347,12786,350167,118798,0.072385


,total faculty,international faculty,number of students,international students,ratio
region,,,,,
Oceania,25347,12786,350167,118798,0.339261
Europe,223271,68267,1991280,454293,0.228141
North America,182123,43836,1546353,292116,0.188906
Africa,1733,379,19593,3325,0.169703
Asia,106734,25462,807003,110100,0.136431
Latin America,43126,5521,396902,34737,0.087520


In [17]:
end = datetime.datetime.now()
print(end - start)      

0:00:50.299779


In [18]:
r = requests.get('https://www.timeshighereducation.com/sites/default/files/the_data_rankings/world_university_rankings_2018_limit0_369a9045a203e176392b9fb8f8c1cb2a.json')
json = r.json()['data']

In [81]:
df_2 = pd.DataFrame(json)
df_2 = df_2[['aliases','name','location','rank','stats_number_students','stats_pc_intl_students','stats_student_staff_ratio']]
df_2 = df_2.head(200)
display(df_2)

,aliases,name,location,rank,stats_number_students,stats_pc_intl_students,stats_student_staff_ratio
0,University of Oxford,University of Oxford,United Kingdom,1,"20,409",38%,11.2
1,University of Cambridge,University of Cambridge,United Kingdom,2,"18,389",35%,10.9
2,California Institute of Technology caltech,California Institute of Technology,United States,=3,"2,209",27%,6.5
3,Stanford University,Stanford University,United States,=3,"15,845",22%,7.5
4,Massachusetts Institute of Technology,Massachusetts Institute of Technology,United States,5,"11,177",34%,8.7
5,Harvard University,Harvard University,United States,6,"20,326",26%,8.9
6,Princeton University,Princeton University,United States,7,"7,955",24%,8.3
7,Imperial College London,Imperial College London,United Kingdom,8,"15,857",55%,11.4
8,University of Chicago,University of Chicago,United States,9,"13,525",25%,6.2
9,ETH Zurich – Swiss Federal Institute of Techno...,ETH Zurich – Swiss Federal Institute of Techno...,Switzerland,=10,"19,233",38%,14.6


In [82]:
def format_rank(x):
    return int(x.replace('=',''))

def format_percentage_students(x):
    return int(x.replace('%','')) /100

def format_number_students(x):
    return int(x.replace(',',''))

In [83]:
# TODO: Care for the rank if tie

df_2['stats_pc_intl_students'] = df_2['stats_pc_intl_students'].apply(lambda x: format_percentage_student(x))
df_2['rank'] = df_2['rank'].apply(lambda x: format_rank(x))
df_2['stats_number_students'] = df_2['stats_number_students'].apply(lambda x: format_number_students(x))
df_2['stats_student_staff_ratio'] = df_2['stats_student_staff_ratio'].apply(pd.to_numeric)


df_2['international students'] = df_2['stats_number_students'] * df_2['stats_pc_intl_students']
df_2['total faculty'] = (df_2['stats_number_students'] / df_2['stats_student_staff_ratio']).astype(int)
df_2 = df_2.drop(['stats_pc_intl_students','stats_student_staff_ratio'],1)

,aliases,name,location,rank,stats_number_students,stats_pc_intl_students,stats_student_staff_ratio,international students,total faculty
0,University of Oxford,University of Oxford,United Kingdom,1,20409,0.38,11.2,7755.42,1822
1,University of Cambridge,University of Cambridge,United Kingdom,2,18389,0.35,10.9,6436.15,1687
2,California Institute of Technology caltech,California Institute of Technology,United States,3,2209,0.27,6.5,596.43,339
3,Stanford University,Stanford University,United States,3,15845,0.22,7.5,3485.90,2112
4,Massachusetts Institute of Technology,Massachusetts Institute of Technology,United States,5,11177,0.34,8.7,3800.18,1284
5,Harvard University,Harvard University,United States,6,20326,0.26,8.9,5284.76,2283
6,Princeton University,Princeton University,United States,7,7955,0.24,8.3,1909.20,958
7,Imperial College London,Imperial College London,United Kingdom,8,15857,0.55,11.4,8721.35,1390
8,University of Chicago,University of Chicago,United States,9,13525,0.25,6.2,3381.25,2181
9,ETH Zurich – Swiss Federal Institute of Techno...,ETH Zurich – Swiss Federal Institute of Techno...,Switzerland,10,19233,0.38,14.6,7308.54,1317


In [86]:
df_2 = df_2.rename(columns={'stats_number_students': 'number of students'})

display(df_2)

,aliases,name,location,rank,number of students,international students,total faculty
0,University of Oxford,University of Oxford,United Kingdom,1,20409,7755.42,1822
1,University of Cambridge,University of Cambridge,United Kingdom,2,18389,6436.15,1687
2,California Institute of Technology caltech,California Institute of Technology,United States,3,2209,596.43,339
3,Stanford University,Stanford University,United States,3,15845,3485.90,2112
4,Massachusetts Institute of Technology,Massachusetts Institute of Technology,United States,5,11177,3800.18,1284
5,Harvard University,Harvard University,United States,6,20326,5284.76,2283
6,Princeton University,Princeton University,United States,7,7955,1909.20,958
7,Imperial College London,Imperial College London,United Kingdom,8,15857,8721.35,1390
8,University of Chicago,University of Chicago,United States,9,13525,3381.25,2181
9,ETH Zurich – Swiss Federal Institute of Techno...,ETH Zurich – Swiss Federal Institute of Techno...,Switzerland,10,19233,7308.54,1317


In [87]:
display_ratio(df_2)

,aliases,name,location,rank,number of students,international students,total faculty,ratio
105,Vanderbilt University,Vanderbilt University,United States,105,12011,1561.43,3639,0.302972
109,University of Copenhagen,University of Copenhagen,Denmark,109,30395,4255.30,7413,0.243889
12,Johns Hopkins University,Johns Hopkins University,United States,13,15498,3719.52,3604,0.232546
11,Yale University,Yale University,United States,12,12155,2552.55,2826,0.232497
153,University of Rochester,University of Rochester,United States,153,9636,2794.44,2240,0.232462
97,Emory University,Emory University,United States,98,12649,2403.31,2874,0.227212
16,Duke University,Duke University,United States,17,15256,3356.32,3390,0.222208
114,École Polytechnique,École Polytechnique,France,115,2799,1007.64,548,0.195784
183,Scuola Normale Superiore di Pisa,Scuola Normale Superiore di Pisa,Italy,184,515,36.05,99,0.192233
185,University of Miami,University of Miami,United States,186,15860,2537.60,2688,0.169483


,aliases,name,location,rank,number of students,international students,total faculty,ratio
24,London School of Economics and Political Science,London School of Economics and Political Science,United Kingdom,25,10065,7146.15,825,0.71
178,University of Luxembourg,University of Luxembourg,Luxembourg,179,4969,2832.33,340,0.57
7,Imperial College London,Imperial College London,United Kingdom,8,15857,8721.35,1390,0.55
37,École Polytechnique Fédérale de Lausanne,École Polytechnique Fédérale de Lausanne,Switzerland,38,9928,5460.40,886,0.55
102,Maastricht University,Maastricht University,Netherlands,103,16727,8363.50,929,0.50
15,University College London ucl,University College London,United Kingdom,16,30304,14848.96,2886,0.49
143,University of St Andrews,University of St Andrews,United Kingdom,143,9005,4322.40,692,0.48
23,Carnegie Mellon University,Carnegie Mellon University,United States,24,12676,5704.20,938,0.45
120,Queen Mary University of London,Queen Mary University of London,United Kingdom,121,15405,6932.25,1158,0.45
39,University of Hong Kong HKU hku The University...,University of Hong Kong,Hong Kong,40,18364,7712.88,1020,0.42


In [89]:
display_ratio(df_2.groupby('location').sum())

,rank,number of students,international students,total faculty,ratio
location,,,,,
Denmark,371,64110,9326.86,10596,0.165278
Russian Federation,194,29236,6431.92,4004,0.136954
Japan,120,48481,4398.48,6464,0.133331
Italy,339,1205,132.65,157,0.130290
Switzerland,666,107852,32745.97,10048,0.093165
United States,4794,1705533,280364.88,152317,0.089308
Taiwan,198,31758,2540.64,2761,0.086939
South Africa,171,20775,3739.50,1775,0.085439
China,839,253911,22882.57,21388,0.084234


,rank,number of students,international students,total faculty,ratio
location,,,,,
Luxembourg,179,4969,2832.33,340,0.570000
United Kingdom,3428,596449,213059.74,44425,0.357214
Hong Kong,443,77663,25158.79,4140,0.323948
Australia,616,268630,83811.65,9937,0.311997
Singapore,74,56101,17085.29,3364,0.304545
Switzerland,666,107852,32745.97,10048,0.303619
New Zealand,192,30348,8800.92,1614,0.290000
Ireland,117,16157,4362.39,708,0.270000
Austria,165,35375,9197.50,1700,0.260000


In [102]:
from incf.countryutils import transformations

transformations.cca2_to_ccn('de')

SyntaxError: invalid syntax (transformations.py, line 151)